In [27]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.preprocessing import OneHotEncoder,OrdinalEncoder,StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [28]:
df = pd.read_csv('loan_data.csv')
df

,person_age,person_gender,person_education,person_income,person_emp_exp,person_home_ownership,loan_amnt,loan_intent,loan_int_rate,loan_percent_income,cb_person_cred_hist_length,credit_score,previous_loan_defaults_on_file,loan_status
0,22.0,female,Master,71948.0,0,RENT,35000.0,PERSONAL,16.02,0.49,3.0,561,No,1
1,21.0,female,High School,12282.0,0,OWN,1000.0,EDUCATION,11.14,0.08,2.0,504,Yes,0
2,25.0,female,High School,12438.0,3,MORTGAGE,5500.0,MEDICAL,12.87,0.44,3.0,635,No,1
3,23.0,female,Bachelor,79753.0,0,RENT,35000.0,MEDICAL,15.23,0.44,2.0,675,No,1
4,24.0,male,Master,66135.0,1,RENT,35000.0,MEDICAL,14.27,0.53,4.0,586,No,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44995,27.0,male,Associate,47971.0,6,RENT,15000.0,MEDICAL,15.66,0.31,3.0,645,No,1
44996,37.0,female,Associate,65800.0,17,RENT,9000.0,HOMEIMPROVEMENT,14.07,0.14,11.0,621,No,1
44997,33.0,male,Associate,56942.0,7,RENT,2771.0,DEBTCONSOLIDATION,10.02,0.05,10.0,668,No,1
44998,29.0,male,Bachelor,33164.0,4,RENT,12000.0,EDUCATION,13.23,0.36,6.0,604,No,1


Without Pipeline

In [29]:
# Make sure the first step is always train_test_split before Preprocessing somehting else for data leaking problems 
X = df.drop(columns = ['loan_status'])
y = df['loan_status']

In [48]:
X_train , X_test , y_train , y_test = train_test_split ( X , y , test_size = 0.2 , random_state = 42 )

In [31]:
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((36000, 13), (9000, 13), (36000,), (9000,))

In [32]:
df.columns

Index(['person_age', 'person_gender', 'person_education', 'person_income',
       'person_emp_exp', 'person_home_ownership', 'loan_amnt', 'loan_intent',
       'loan_int_rate', 'loan_percent_income', 'cb_person_cred_hist_length',
       'credit_score', 'previous_loan_defaults_on_file', 'loan_status'],
      dtype='object')

In [33]:
df.person_education.unique()

array(['Master', 'High School', 'Bachelor', 'Associate', 'Doctorate'],
      dtype=object)

In [ ]:
categories_without_order = ['person_gender','person_home_ownership','loan_intent','previous_loan_defaults_on_file']
categories_with_order = ['person_education']
person_education_order = [ [  'High School', 'Bachelor', 'Master', 'Associate', 'Doctorate' ] ]

In [ ]:
column_transformer1 = ColumnTransformer(
                                            transformers = [
                                                               ( 'ohe', OneHotEncoder(sparse_output=False , drop = 'first' ) , categories_without_order ),
                                                               ( 'ord', OrdinalEncoder(categories = person_education_order ) , categories_with_order ),
                                                               ('scale_num', StandardScaler(), ['loan_amnt','person_income'])
                                            ] ,
                                            remainder = 'passthrough'
)

In [38]:
X_train_tranformed = column_transformer1.fit_transform(X_train)
X_test_tranformed = column_transformer1.transform(X_test)

In [39]:
X_train_tranformed,X_test_tranformed

(array([[0.00e+00, 0.00e+00, 0.00e+00, ..., 1.50e-01, 9.00e+00, 6.31e+02],
        [1.00e+00, 0.00e+00, 0.00e+00, ..., 1.60e-01, 4.00e+00, 6.59e+02],
        [0.00e+00, 0.00e+00, 0.00e+00, ..., 7.00e-02, 1.10e+01, 6.83e+02],
        ...,
        [0.00e+00, 0.00e+00, 0.00e+00, ..., 1.80e-01, 9.00e+00, 6.52e+02],
        [1.00e+00, 0.00e+00, 0.00e+00, ..., 1.10e-01, 3.00e+00, 6.04e+02],
        [0.00e+00, 0.00e+00, 0.00e+00, ..., 7.00e-02, 3.00e+00, 6.64e+02]],
       shape=(36000, 19)),
 array([[1.00e+00, 0.00e+00, 0.00e+00, ..., 8.00e-02, 1.00e+01, 6.01e+02],
        [1.00e+00, 0.00e+00, 0.00e+00, ..., 1.60e-01, 4.00e+00, 6.47e+02],
        [0.00e+00, 0.00e+00, 0.00e+00, ..., 1.30e-01, 4.00e+00, 7.11e+02],
        ...,
        [0.00e+00, 0.00e+00, 0.00e+00, ..., 1.80e-01, 3.00e+00, 6.14e+02],
        [1.00e+00, 0.00e+00, 0.00e+00, ..., 2.20e-01, 1.40e+01, 6.95e+02],
        [0.00e+00, 0.00e+00, 0.00e+00, ..., 2.40e-01, 3.00e+00, 6.35e+02]],
       shape=(9000, 19)))

In [40]:
lor = LogisticRegression(max_iter=1000)
lor.fit(X_train_tranformed,y_train)
# The error belowe just says that we should try more iterations due to number of rows , but that just means we should have done scalling from the start.

c:\Users\hp\Desktop\ML\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 1000 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=1000).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,1000
,multi_class,'deprecated'


In [41]:
y_pred = lor.predict(X_test_tranformed)
y_pred

array([0, 0, 1, ..., 1, 1, 1], shape=(9000,))

In [42]:
y_test

37979    0
9911     0
43386    1
13822    0
44810    1
        ..
14572    0
30384    0
36665    0
30569    0
5321     0
Name: loan_status, Length: 9000, dtype: int64

In [43]:
scores = cross_val_score(lor , X_train_tranformed , y_train , cv = 10, scoring = 'accuracy' ).mean()
scores

c:\Users\hp\Desktop\ML\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 1000 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=1000).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\hp\Desktop\ML\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 1000 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=1000).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Pl

np.float64(0.8956111111111114)

Now we will do all these steps with the help of the pipeline

In [ ]:
# We have to repeat all the steps till the column transformer and make sure we dont fit/transform anything.

X = df.drop(columns = ['loan_status'])
y = df['loan_status']
X_train , X_test , y_train , y_test = train_test_split ( X , y , test_size = 0.2 , random_state = 42 )
categories_without_order = ['person_gender','person_home_ownership','loan_intent','previous_loan_defaults_on_file']
categories_with_order = ['person_education']
person_education_order = [ [  'High School', 'Bachelor', 'Master', 'Associate', 'Doctorate' ] ]
column_transformer1 = ColumnTransformer(
                                            transformers = [
                                                               ( 'ohe', OneHotEncoder(sparse_output=False , drop = 'first' ) , categories_without_order ),
                                                               ( 'ord', OrdinalEncoder(categories = person_education_order ) , categories_with_order ),
                                                               ('scale_num', StandardScaler(), ['loan_amnt','person_income'])
                                            ] ,
                                            remainder = 'passthrough'
)



# Building pipeline
pipe = Pipeline(
                [
                    ( 'preprocessor' , column_transformer1 ),
                    (  'logistic_regression_model' , LogisticRegression( max_iter = 1000 ) )
                ]
)

In [46]:
# Fitting pipeline 
pipe.fit(X_train,y_train)

#Making predictions
y_pred = pipe.predict(X_test)

c:\Users\hp\Desktop\ML\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 1000 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=1000).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [47]:
# Now lets evaluate the same 
scores = cross_val_score(lor , X_train_tranformed , y_train , cv = 5, scoring = 'accuracy' ).mean()
scores

c:\Users\hp\Desktop\ML\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 1000 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=1000).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\hp\Desktop\ML\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 1000 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=1000).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Pl

np.float64(0.8950833333333333)